# From user stories to feature models
## Context
The Software Product Lines (SPL) are now a well-established way to efficiently produce highly-configurable software products. However, migrating an existing software family into an integrated SPL platform is still challenging and organizations may be reluctant to adopt the approach. Hesitations often arise from the lack of standard procedures guiding the process, as well as doubts on the cost/benefit ratio. Although companies own large bases of well-documented code, they still manually build tailored applications from the base code to meet their clients' requirements, sometimes with a clone-and-own strategy.

One of the difficulties during the migration process is to extract a relevant feature model from the existing products. In this notebook, we address this difficulty in the context of products built with an agile process, specifically where requirements are defined using user stories, and a version control system  is used to store and trace project artifacts. User stories are nowadays widely used in a large part of software projects to define requirements. VCS platforms like GitLab or Github are widespread support not only to manage code artifacts but also as a central part of project management, directly using integrated tools to manage the user stories or integrating external tools in the VCS platform. Therefore, our approach is suitable for a large part of projects.

In this context, we have developed an automated feature model synthesis approach based on the knowledge collected from the user stories and the way they are linked to the product code through traceability links established in the VCS.


### Rough view: A Step in migrating a set of similar software into a Soft. product line

**From existing products**  
![fromExistingProducts.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/fromExistingProducts.png)  


**To a feature model**  
![ToAFeatureModel.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/ToAFeatureModel.png)  

## Readme
**Objective**   
Provide an automated process for feature models synthesis using Formal Concept Analysis and Relational Concept Analysis by leveraging User-stories and code merges.
All the source code is available in a [Github repository](https://github.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM)

**Inputs:**  
- Dataset extracted from Github or Gitlab:
  - User-stories title and request anwser
  - The user-stories related products
  - The user-stories related merges
  - The merges related changes
  - The changes related Files
  - The changes related Code Diff
  
**Outputs:**  
Two kinds of feature representations:
- *Feature model*: a hierarchical diagram that visually illustrates the features, and their dependencies
- *Feature role model*: a feature model hierarchized by roles

**Process:**  
User-stories are used to identify the roles, features and create abstract features. They are the nodes and the leafs on the outputs models.  
The relations between the nodes and leafs are computed with FCA and RCA.

**Big picture**  
![processCono.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/processCono.png)

**From User stories to logical constraints**  
![UStoFM.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/UStoFM.png)



**FCA and RCA Model**  
<div>
<img src="https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/USFeatureFCA.png" width="800"/>
</div>

**Formal Concept Analysis:**  
Each object from Version Control System is represented with a Formal Context:   
- User-stories
- Products
- Roles
- Features
- Merges
- Changes
- Code diffs
- Files
- Abstract features

**Relational Concept Analysis**  
The relations between each objects are in their related Relational Concepts:  
- Products x User-stories  
- User-stories x Roles
- User-stories x Features
- Features x Abstract features
- User-stories x Merges
- Changes x Files
- Changes x Code diffs

**Tools:**
- [FCA4J](https://www.lirmm.fr/fca4j/Introduction.html) to perform Formal concept analysis
- [RCFT](https://www.lirmm.fr/fca4j/Family.html) file extension for FCA and RCA data representation
- [FeatureIDE](https://featureide.github.io/) for Feature Models vizualisation
- Elbows, Kmeans and Word2Vec for Clustering 

### User stories per products in our example:

| usID     | User Stories                                                                                         | Producs |
|----------|--------------------------------------------------------------------------------------------| -- |
| 1        | As a farmer , I can refresh the predicted weather                                          | ["Vine","Orchard"] 	
| 2        | As a farmer, I can CRUD plots                                                              | ["Vine","Orchard","Almond"]	
| 3        | As a farmer with a plot, I can edit the parameters of a plot (current season)              | ["Vine","Orchard"]
| 4        | As a farmer with a plot, I can sort my plots in the list                                   | ["Orchard"]
| 5        | As a farmer with a plot, I can filter my plots in the list                                 | ["Orchard"]
| 6        | As a farmer with a plot, I can export observation data for a plot                          | ["Vine"]
| 7        | As a farmer with a plot, I know when my plots will be in danger                            | ["Vine","Orchard"]
| 8        | As a farmer irrigator, I can manage my irrigations and recommendations in my favorite unit | ["Almond"]
| 9        | As a farmer irrigator, I choose my preferred irrigation unit in my user-settings           | ["Vine"]
| 10       | As a farmer irrigator, I can view my irrigation recommendations in my favorite unit        | ["Vine","Almond"]
| 11       | As a admin, I can CRUD a farmer                                                            | ["Vine","Orchard"]
| 12       | As a admin, I can relaunch all failed simulation                                           | ["Vine","Orchard","Almond"]


### Outputs visualized:
#### Feature Model with FeatureIDE:
![exampleFMCONO.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/exampleFMCONO.png)

#### Feature Role Model with FeatureIDE:
![exampleFRMCONO.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/exampleFRMCONO.png)

## For googleColab, git clone

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    !apt-get install git

    !git clone https://github.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM.git
    !cp -a CoNoConcepts_ThomasGEORGES_LIRMM/. /content
except:
    print("Not running in googlecolab")

Not running in googlecolab


## Imports

In [2]:
import pandas as pd # Dataframe for data manipulation
pd.set_option('display.colheader_justify', 'center')
pd.options.display.max_colwidth = 100

import nltk # Natural langage toolkit
nltk.download('punkt', quiet=True)

import numpy as np # Matrix manipulation
import subprocess # to launch bash commands
import re # For Regexs
import gensim.downloader as api # Topic modeling library. Provide pre-trained models (e.g. Word2Vec)
import json # Json manipulation


In [3]:
# Customs fonction for clustering
from ClusteringUtils import *

## Dataset

US_title 	US_data 	products 	MR_data 	nb_mr 	changes

In [4]:
df_complet = pd.read_csv("datasetUSExample.csv").drop("Unnamed: 0", axis=1)
df_complet[df_complet["nb_mr"] > 0].fillna("").dropna()
df_complet

,US_title,US_data,products,MR_data,nb_mr,changes
0,"As a farmer , I can refresh the predicted weather","[{""id"":1,""iid"":1,""project_id"":1,""title"":""As a farmer, I can refresh the predicted weather"",""labe...","[""Vine"",""Orchard""]","[{""id"": 1}]",1,"[{""id"":1,""changes"":[{""old_path"":""src/farmer"",""new_path"":""src/farmer"",""new_file"":""False"",""renamed..."
1,"As a farmer, I can CRUD plots","[{""id"":2,""iid"":2,""project_id"":1,""title"":""As a farmer, I can CRUD plots"",""labels"":[""Vine"",""Orchar...","[""Vine"",""Orchard"",""Almond""]","[{""id"": 2}]",1,"[{""id"":2,""changes"":[{""old_path"":""src/farmer"",""new_path"":""src/farmer"",""new_file"":""False"",""renamed..."
2,"As a farmer with a plot, I can edit the parameters of a plot (current season)","[{""id"":3,""iid"":3,""project_id"":1,""title"":""As a farmer with a plot, I can edit the parameters of a...","[""Vine"",""Orchard""]","[{""id"": 3}]",1,"[{""id"":3,""changes"":[{""old_path"":""src/farmer"",""new_path"":""src/farmer"",""new_file"":""False"",""renamed..."
3,"As a farmer with a plot, I can sort my plots in the list","[{""id"":4,""iid"":4,""project_id"":1,""title"":""As a farmer with a plot, I can sort my plots in the lis...","[""Orchard""]","[{""id"": 4}]",1,"[{""id"":4,""changes"":[{""old_path"":""src/plotList"",""new_path"":""src/plotList"",""new_file"":""False"",""ren..."
4,"As a farmer with a plot, I can filter my plots in the list","[{""id"":5,""iid"":5,""project_id"":1,""title"":""As a farmer with a plot, I can filter my plots in the l...","[""Orchard""]","[{""id"": 5}]",1,"[{""id"":5,""changes"":[{""old_path"":""src/plotList"",""new_path"":""src/plotList"",""new_file"":""True"",""rena..."
5,"As a farmer with a plot, I can export observation data for a plot","[{""id"":6,""iid"":6,""project_id"":1,""title"":""As a farmer with a plot, I can export observation data ...","[""Vine""]","[{""id"": 6}]",1,"[{""id"":6,""changes"":[{""old_path"":""src/export"",""new_path"":""src/export"",""new_file"":""True"",""renamed_..."
6,"As a farmer with a plot, I know when my plots will be in danger","[{""id"":7,""iid"":7,""project_id"":1,""title"":""As a farmer with a plot, I know when my plots will be i...","[""Vine"",""Orchard""]","[{""id"": 7}]",1,"[{""id"":7,""changes"":[{""old_path"":""src/danger"",""new_path"":""src/danger"",""new_file"":""True"",""renamed_..."
7,"As a farmer irrigator, I can manage my irrigations and recommendations in my favorite unit","[{""id"":8,""iid"":8,""project_id"":1,""title"":""As a farmer irrigator, I can manage my irrigations and ...","[""Almond""]","[{""id"": 8}]",1,"[{""id"":8,""changes"":[{""old_path"":""src/irrigation"",""new_path"":""src/irrigation"",""new_file"":""True"",""..."
8,"As a farmer irrigator, I choose my preferred irrigation unit in my user-settings","[{""id"":9,""iid"":9,""project_id"":1,""title"":""As a farmer irrigator, I choose my preferred irrigation...","[""Vine""]","[{""id"": 9}]",1,"[{""id"":9,""changes"":[{""old_path"":""src/irrigation"",""new_path"":""src/irrigation"",""new_file"":""False"",..."
9,"As a farmer irrigator, I can view my irrigation recommendations in my favorite unit","[{""id"":10,""iid"":10,""project_id"":1,""title"":""As a farmer irrigator, I can view my irrigation recom...","[""Vine"",""Almond""]","[{""id"": 10}]",1,"[{""id"":10,""changes"":[{""old_path"":""src/irrigation"",""new_path"":""src/irrigation"",""new_file"":""False""..."


## Formal Contexts
Each object from Version Control System is represented with a Formal Context:
- User-stories
- Products
- Roles
- Features
- Merges
- Changes
- Code diffs
- Files
- Abstract features


In [5]:
# Union between two list
def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

# role uniformisation
def uniformRoles(roles):
    roles = list(map(lambda x: x.replace('admin', 'administrator'), roles))
    return roles

# Cleaning role
def cleanRole(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace("As ", "")
    sentence = sentence.replace("as ", "")
    sentence = sentence.replace("a ", "")
    sentence = sentence.replace("an ", "")
    sentence = sentence.replace("( ", "")
    sentence = sentence.replace(") ", "")
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(" admin", "administrator")
    sentence = sentence.replace(" ", "")
    return sentence

# Cleaning Feature
def cleanFeature(sentence):
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace("(", " ")
    sentence = sentence.replace(")", " ")
    sentence = sentence.replace("  ", " ")
    return sentence[1:]

### User-stories

In [6]:
df_US = pd.DataFrame(data={"US": df_complet["US_title"]})
df_US["US"] = df_US["US"].apply(lambda x: x.replace(",", ""))
df_US.to_csv("CSVs/ctxF_" + "userStories" + "_Example.csv")
df_US

,US
0,As a farmer I can refresh the predicted weather
1,As a farmer I can CRUD plots
2,As a farmer with a plot I can edit the parameters of a plot (current season)
3,As a farmer with a plot I can sort my plots in the list
4,As a farmer with a plot I can filter my plots in the list
5,As a farmer with a plot I can export observation data for a plot
6,As a farmer with a plot I know when my plots will be in danger
7,As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit
8,As a farmer irrigator I choose my preferred irrigation unit in my user-settings
9,As a farmer irrigator I can view my irrigation recommendations in my favorite unit


### Products

In [7]:
listproduits = df_complet["products"]
productPerUs = []
listTmp = []
for elem in listproduits:
    productPerUs.append(elem)
    for prod in elem[2:-2].replace("\"", "").split(","):
        listTmp = union(listTmp, [prod])
listproduits = listTmp

#  All products without duplicate
df_products = pd.DataFrame(data={"products": listproduits})
df_products.to_csv("CSVs/ctxF_" + "products" + "_Example.csv")
df_products

,products
0,Almond
1,Vine
2,Orchard


In [8]:
## [Product] per US
df_productsPerUS = pd.DataFrame(data={"products": productPerUs}).apply(lambda x : x.replace("\"",""))
df_productsPerUS

,products
0,"[""Vine"",""Orchard""]"
1,"[""Vine"",""Orchard"",""Almond""]"
2,"[""Vine"",""Orchard""]"
3,"[""Orchard""]"
4,"[""Orchard""]"
5,"[""Vine""]"
6,"[""Vine"",""Orchard""]"
7,"[""Almond""]"
8,"[""Vine""]"
9,"[""Vine"",""Almond""]"


### Roles

In [9]:
roleData = []
for us in df_complet["US_title"]:
    usSplit = us.split("I")
    roleData.append(cleanRole(usSplit[0]))
roleData = uniformRoles(roleData)


df_roles = pd.DataFrame(data={"roles": list(dict.fromkeys(roleData))})
df_roles.to_csv("CSVs/ctxF_" + "roles" + "_Example.csv")
df_roles

,roles
0,farmer
1,farmerwithplot
2,farmerirrigator
3,administrator


In [10]:
df_rolesPerUS = pd.DataFrame(data={"roles": roleData})
df_rolesPerUS

,roles
0,farmer
1,farmer
2,farmerwithplot
3,farmerwithplot
4,farmerwithplot
5,farmerwithplot
6,farmerwithplot
7,farmerirrigator
8,farmerirrigator
9,farmerirrigator


### Features

In [11]:
featureData = []
for us in df_complet["US_title"]:
    usSplit = us.split("I")
    featureData.append(cleanFeature(("JJ" + "j".join(usSplit[1:]))[2:]))
df_features = pd.DataFrame(data={"features": featureData})
df_features

,features
0,can refresh the predicted weather
1,can CRUD plots
2,can edit the parameters of a plot current season
3,can sort my plots in the list
4,can filter my plots in the list
5,can export observation data for a plot
6,know when my plots will be in danger
7,can manage my irrigations and recommendations in my favorite unit
8,choose my preferred irrigation unit in my user-settings
9,can view my irrigation recommendations in my favorite unit


In [12]:
len(list(dict.fromkeys(df_features["features"].tolist())))


df_featuresPerUS = pd.DataFrame(data={"features": list(dict.fromkeys(df_features["features"].tolist()))})
df_featuresPerUS.to_csv("CSVs/ctxF_" + "features" + "_Example.csv")
df_featuresPerUS

,features
0,can refresh the predicted weather
1,can CRUD plots
2,can edit the parameters of a plot current season
3,can sort my plots in the list
4,can filter my plots in the list
5,can export observation data for a plot
6,know when my plots will be in danger
7,can manage my irrigations and recommendations in my favorite unit
8,choose my preferred irrigation unit in my user-settings
9,can view my irrigation recommendations in my favorite unit


### Merges

In [13]:
mrList = []
usList = []
df_data = df_complet[df_complet["nb_mr"] > 0].fillna("").dropna()
for index, row in df_data.iterrows():
    for specMR in eval(row["MR_data"]):
            mrList.append(specMR["id"])
            usList.append(row["US_title"])

df_MRs = pd.DataFrame(data={"mrs":mrList})
df_MRs.to_csv("CSVs/ctxF_" + "mrs" + "_Example.csv")
df_MRs

,mrs
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [14]:
df_MRsPerUS = pd.DataFrame(data={"mrs":usList})
df_MRsPerUS

,mrs
0,"As a farmer , I can refresh the predicted weather"
1,"As a farmer, I can CRUD plots"
2,"As a farmer with a plot, I can edit the parameters of a plot (current season)"
3,"As a farmer with a plot, I can sort my plots in the list"
4,"As a farmer with a plot, I can filter my plots in the list"
5,"As a farmer with a plot, I can export observation data for a plot"
6,"As a farmer with a plot, I know when my plots will be in danger"
7,"As a farmer irrigator, I can manage my irrigations and recommendations in my favorite unit"
8,"As a farmer irrigator, I choose my preferred irrigation unit in my user-settings"
9,"As a farmer irrigator, I can view my irrigation recommendations in my favorite unit"


### Changes

In [15]:
hashChangesList = []
idMrList = []
for index,row in df_complet[df_complet["nb_mr"] > 0].fillna("").dropna().iterrows():
    for mr in eval(row["changes"]):
        listTmp = []
        for change in mr["changes"]:
            hashChangesList.append(hash(change["diff"]))
            listTmp.append(hash(change["diff"]))
        idMrList.append(listTmp)
            

df_changes = pd.DataFrame(data={"changes":hashChangesList})
df_changes.to_csv("CSVs/ctxF_" + "changes" + "_Example.csv")
df_changes

,changes
0,-7467257910948230577
1,-2369461973268875310
2,4830427670682130528
3,4532039579069182754
4,-3247514657019673527
5,-6143752595892114764
6,7361577847672698194
7,-606811720574785494
8,8505594899342993216
9,1522805805735262404


In [16]:
df_changesPerMR = pd.DataFrame(data={"changes":idMrList})
df_changesPerMR

,changes
0,"[-7467257910948230577, -2369461973268875310]"
1,"[4830427670682130528, 4532039579069182754]"
2,"[-3247514657019673527, -6143752595892114764]"
3,"[7361577847672698194, -606811720574785494]"
4,"[8505594899342993216, 1522805805735262404]"
5,"[-6072351681716626404, -1901426545407485672]"
6,"[4901052965618317889, -2094431102868343337]"
7,"[3661700103020512783, -5476404769532837106]"
8,"[-6935385260835073677, -4598298317307740577]"
9,"[-2317302283732295464, -509010681288107727]"


### Files

In [17]:
filenamesList = []
filenamesListDuplicate = []
filenamesPerChanges = []
for row in df_complet[df_complet["nb_mr"] > 0].fillna("").dropna()["changes"]:
    for mr in eval(row):
        filenamesListDuplicateList = []
        for change in mr["changes"]:             
            filenamesList = union(filenamesList,[change["new_path"]])
            filenamesListDuplicateList.append(change["new_path"])
            filenamesPerChanges.append(change["new_path"])
        filenamesListDuplicate.append(filenamesListDuplicateList)

df_filenames = pd.DataFrame(data={"filenames":filenamesList})
df_filenamesPerMRs = pd.DataFrame(data={"filenames":filenamesListDuplicate})
df_filenamesPerChanges = pd.DataFrame(data={"filenames":filenamesPerChanges})
df_filenames.to_csv("CSVs/ctxF_" + "filenames" + "_Example.csv")
df_filenames

,filenames
0,src/user-settings
1,src/alert
2,src/plotList
3,src/danger
4,src/observation
5,src/export
6,src/admin
7,src/weather
8,src/recommendation
9,src/plot


In [18]:
df_filenamesPerChanges

,filenames
0,src/farmer
1,src/weather
2,src/farmer
3,src/plot
4,src/farmer
5,src/plot
6,src/plotList
7,src/plot
8,src/plotList
9,src/plot


### Diff

In [19]:
editType = ["new_file",
           "renamed_file",
           "deleted_file", "edited_file"]
df_editTypes = pd.DataFrame(data={"editTypes":editType})
df_editTypes.to_csv("CSVs/ctxF_" + "editTypes" + "_Example.csv")
df_editTypes


,editTypes
0,new_file
1,renamed_file
2,deleted_file
3,edited_file


In [20]:
def getTypeEdit(isRenamed_file, isDeleted_file, isNew_file):
    if (isRenamed_file == str(True)):
        return "renamed_file"
    if (isDeleted_file == str(True)):
        return "deleted_file"
    if (isNew_file == str(True)):
        return "new_file"
    return "edited_file"

In [21]:
editType = []
for index,row in df_complet[df_complet["nb_mr"] > 0].fillna("").dropna().iterrows():
    for mr in eval(row["changes"]):
        for change in mr["changes"]:
            editType.append([getTypeEdit(change["renamed_file"], change["deleted_file"], change["new_file"])])

df_editTypesPerChanges = pd.DataFrame(data={"editTypes":editType})
df_editTypesPerChanges

,editTypes
0,[edited_file]
1,[new_file]
2,[edited_file]
3,[edited_file]
4,[edited_file]
5,[edited_file]
6,[edited_file]
7,[edited_file]
8,[new_file]
9,[edited_file]


### Clustering for Abstract features

- Pre trained model to vectorize *n* user stories
- Elbow method to get the optimal number of clusters *k*
- Kmeans to arrange the *n* user stories in *k* clusters

In [22]:
model = api.load('glove-wiki-gigaword-200')
#model = api.load('glove-wiki-gigaword-300')

In [23]:
# data is a dataframe of array
def ctxRelPerCtxForms(df_ctxFormX, data):

    tabRet = []
    for indexData, rowData in data.iterrows():
        tabContains = []
        
        for indexCtx, rowCtx in df_ctxFormX.iterrows():
            
            if rowCtx[0] in rowData[0]:
                tabContains.append("x")
            else:
                tabContains.append("")
        tabRet.append(tabContains)
    return tabRet

def clusterFeatures(model,features):    
    
    feature_vectors = [document_vector(model, feature) for feature in features]

    nbClusters= getOptimalClusterNumber(features, feature_vectors)
    
    if nbClusters > (len(features)/4)+2:
        nbClusters = int(len(features)/4)+2
    if nbClusters == 0:
        nbClusters = 1
    return getClusterNamed(model, features, nbClusters)
 
    
features2Roles = my_array = np.array(ctxRelPerCtxForms(df_roles, df_rolesPerUS))
df_features2Roles = pd.DataFrame(my_array, columns=df_roles["roles"], index=df_features["features"])
#df_features2Roles


In [24]:
df_features2Roles['roleCluster'] = df_features2Roles.apply(lambda row: min([col for col, val in row.items() if val == 'x'], key=len) if 'x' in row.values else '', axis=1)

featuresRoles = []
for role in df_roles["roles"].tolist():
    feat = df_features2Roles[df_features2Roles['roleCluster'] == role].index.tolist()
    if(len(feat)>0):
        featuresRoles.append(list(map(lambda x: x.split(" "), feat)))
featuresRoles                     
    
featuresPerClusters = []
for featuresPerRoles in featuresRoles:
    featuresPerClusters.append(clusterFeatures(model, featuresPerRoles))

Cluster  0
Cluster  1
Cluster  2
Cluster  3
Cluster  0


In [25]:
df_rolesFeaturesSuperF = pd.DataFrame()
df_rolesFeaturesSuperF["features"] = df_features["features"]
df_rolesFeaturesSuperF["clusters"] = ""
df_rolesFeaturesSuperF= df_rolesFeaturesSuperF.set_index('features')


for clusterNamed in featuresPerClusters:
    for cluster, features in clusterNamed:
        for feature in features:
            df_rolesFeaturesSuperF.loc[feature,"clusters"] = cluster 
df_clusters = df_rolesFeaturesSuperF[["clusters"]]        
df_rolesFeaturesSuperF[["clusters"]].sort_values(by=['clusters'])

,clusters
features,
can CRUD a farmer,crud farmer relaunch
can relaunch all failed simulation,crud farmer relaunch
can CRUD plots,crud plots
can edit the parameters of a plot current season,plots plot list
can sort my plots in the list,plots plot list
can filter my plots in the list,plots plot list
can export observation data for a plot,plots plot list
know when my plots will be in danger,plots plot list
can refresh the predicted weather,refresh predicted weather


In [26]:

df_clusters = pd.DataFrame(data={"clusters":list(dict.fromkeys(df_rolesFeaturesSuperF["clusters"].tolist()))})

df_clusters.to_csv("CSVs/ctxF_" + "AbstractFeature" + "_Example.csv")
df_clusters

,clusters
0,refresh predicted weather
1,crud plots
2,plots plot list
3,unit recommendations favorite
4,crud farmer relaunch


## Relational Contexts
The relations between each objects are in their related Relational Concepts:
- Products x User-stories
- User-stories x Roles
- User-stories x Features
- Features x Abstract features
- User-stories x Merges
- Changes x Files
- Changes x Code diffs


### Product x UserStory : products2UserStories

In [27]:
products2UserStories = my_array = np.array(ctxRelPerCtxForms(df_products, df_productsPerUS))

In [28]:
df_products2UserStories = pd.DataFrame(my_array, columns=df_products["products"], index=df_US["US"])
df_products2UserStories.to_csv("CSVs/ctxR_" + "userStories2products" + "_Example.csv")
df_products2UserStories.transpose()

US,As a farmer I can refresh the predicted weather,As a farmer I can CRUD plots,As a farmer with a plot I can edit the parameters of a plot (current season),As a farmer with a plot I can sort my plots in the list,As a farmer with a plot I can filter my plots in the list,As a farmer with a plot I can export observation data for a plot,As a farmer with a plot I know when my plots will be in danger,As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,As a farmer irrigator I choose my preferred irrigation unit in my user-settings,As a farmer irrigator I can view my irrigation recommendations in my favorite unit,As a admin I can CRUD a farmer,As a admin I can relaunch all failed simulation
products,,,,,,,,,,,,
Almond,,x,,,,,,x,,x,,x
Vine,x,x,x,,,x,x,,x,x,x,x
Orchard,x,x,x,x,x,,x,,,,x,x


In [29]:
products2UserStories = my_array = np.array(ctxRelPerCtxForms(df_products, df_productsPerUS))

In [30]:
df_products2features = pd.DataFrame(my_array, columns=df_products["products"], index=df_features["features"])
df_products2features.to_csv("CSVs/ctxR_" + "products2features" + "_Example.csv")
# Products x features for FM synthesis
df_products2features

products,Almond,Vine,Orchard
features,,,
can refresh the predicted weather,,x,x
can CRUD plots,x,x,x
can edit the parameters of a plot current season,,x,x
can sort my plots in the list,,,x
can filter my plots in the list,,,x
can export observation data for a plot,,x,
know when my plots will be in danger,,x,x
can manage my irrigations and recommendations in my favorite unit,x,,
choose my preferred irrigation unit in my user-settings,,x,


### US x Role : userStories2Roles

In [31]:
userStories2Roles = my_array = np.array(ctxRelPerCtxForms(df_roles, df_rolesPerUS))

In [32]:
df_userStories2Roles = pd.DataFrame(my_array, columns=df_roles["roles"], index=df_US["US"])
df_userStories2Roles.to_csv("CSVs/ctxR_" + "userStories2Roles" + "_Example.csv")
df_userStories2Roles

roles,farmer,farmerwithplot,farmerirrigator,administrator
US,,,,
As a farmer I can refresh the predicted weather,x,,,
As a farmer I can CRUD plots,x,,,
As a farmer with a plot I can edit the parameters of a plot (current season),x,x,,
As a farmer with a plot I can sort my plots in the list,x,x,,
As a farmer with a plot I can filter my plots in the list,x,x,,
As a farmer with a plot I can export observation data for a plot,x,x,,
As a farmer with a plot I know when my plots will be in danger,x,x,,
As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,x,,x,
As a farmer irrigator I choose my preferred irrigation unit in my user-settings,x,,x,


In [33]:
features2Roles = my_array = np.array(ctxRelPerCtxForms(df_roles, df_rolesPerUS))

In [34]:
features2Roles = pd.DataFrame(my_array, columns=df_roles["roles"], index=df_features["features"])
features2Roles.to_csv("CSVs/ctxR_" + "features2Roles" + "_Example.csv")
# Feature x role for FM synthesis
features2Roles

roles,farmer,farmerwithplot,farmerirrigator,administrator
features,,,,
can refresh the predicted weather,x,,,
can CRUD plots,x,,,
can edit the parameters of a plot current season,x,x,,
can sort my plots in the list,x,x,,
can filter my plots in the list,x,x,,
can export observation data for a plot,x,x,,
know when my plots will be in danger,x,x,,
can manage my irrigations and recommendations in my favorite unit,x,,x,
choose my preferred irrigation unit in my user-settings,x,,x,


### US x Feature : userStories2features

In [35]:
userStories2features = my_array = np.array(ctxRelPerCtxForms(df_features, df_featuresPerUS))

In [36]:
df_userStories2features = pd.DataFrame(my_array, columns=df_features["features"], index=df_US["US"])
df_userStories2features.to_csv("CSVs/ctxR_" + "userStories2features" + "_Example.csv")
df_userStories2features

features,can refresh the predicted weather,can CRUD plots,can edit the parameters of a plot current season,can sort my plots in the list,can filter my plots in the list,can export observation data for a plot,know when my plots will be in danger,can manage my irrigations and recommendations in my favorite unit,choose my preferred irrigation unit in my user-settings,can view my irrigation recommendations in my favorite unit,can CRUD a farmer,can relaunch all failed simulation
US,,,,,,,,,,,,
As a farmer I can refresh the predicted weather,x,,,,,,,,,,,
As a farmer I can CRUD plots,,x,,,,,,,,,,
As a farmer with a plot I can edit the parameters of a plot (current season),,,x,,,,,,,,,
As a farmer with a plot I can sort my plots in the list,,,,x,,,,,,,,
As a farmer with a plot I can filter my plots in the list,,,,,x,,,,,,,
As a farmer with a plot I can export observation data for a plot,,,,,,x,,,,,,
As a farmer with a plot I know when my plots will be in danger,,,,,,,x,,,,,
As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,,,,,,,,x,,,,
As a farmer irrigator I choose my preferred irrigation unit in my user-settings,,,,,,,,,x,,,


### Feature x AbstractF : features2AbstracFeatures

In [37]:
features2AbstracFeatures = my_array = np.array(ctxRelPerCtxForms(df_clusters, df_rolesFeaturesSuperF[["clusters"]]))

In [38]:
df_features2AbstracFeatures = pd.DataFrame(my_array, columns=df_clusters["clusters"], index=df_features["features"])
df_features2AbstracFeatures.to_csv("CSVs/ctxR_" + "features2AbstracFeatures" + "_Example.csv")
df_features2AbstracFeatures.transpose().sort_values(by="clusters").transpose()

clusters,crud farmer relaunch,crud plots,plots plot list,refresh predicted weather,unit recommendations favorite
features,,,,,
can refresh the predicted weather,,,,x,
can CRUD plots,,x,,,
can edit the parameters of a plot current season,,,x,,
can sort my plots in the list,,,x,,
can filter my plots in the list,,,x,,
can export observation data for a plot,,,x,,
know when my plots will be in danger,,,x,,
can manage my irrigations and recommendations in my favorite unit,,,,,x
choose my preferred irrigation unit in my user-settings,,,,,x


In [39]:
features2AbstracFeatures = my_array = np.array(ctxRelPerCtxForms(df_clusters, df_rolesFeaturesSuperF[["clusters"]]))

In [40]:
df_us2AbstracFeatures = pd.DataFrame(my_array, columns=df_clusters["clusters"], index=df_US["US"])
df_us2AbstracFeatures.to_csv("CSVs/ctxR_" + "us2AbstracFeatures" + "_Example.csv")
df_us2AbstracFeatures.transpose().sort_values(by="clusters").transpose()

clusters,crud farmer relaunch,crud plots,plots plot list,refresh predicted weather,unit recommendations favorite
US,,,,,
As a farmer I can refresh the predicted weather,,,,x,
As a farmer I can CRUD plots,,x,,,
As a farmer with a plot I can edit the parameters of a plot (current season),,,x,,
As a farmer with a plot I can sort my plots in the list,,,x,,
As a farmer with a plot I can filter my plots in the list,,,x,,
As a farmer with a plot I can export observation data for a plot,,,x,,
As a farmer with a plot I know when my plots will be in danger,,,x,,
As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,,,,,x
As a farmer irrigator I choose my preferred irrigation unit in my user-settings,,,,,x


### US x MR : df_userStories2mrs

In [41]:
userStories2mrs = my_array = np.array(ctxRelPerCtxForms(df_US, df_MRsPerUS))

In [42]:
df_userStories2mrs = pd.DataFrame(my_array, columns=df_US["US"], index=df_MRs["mrs"])
df_userStories2mrs.to_csv("CSVs/ctxR_" + "userStories2mrs" + "_Example.csv")
df_userStories2mrs = df_userStories2mrs.transpose()
df_userStories2mrs

mrs,1,2,3,4,5,6,7,8,9,10,11,12
US,,,,,,,,,,,,
As a farmer I can refresh the predicted weather,,,,,,,,,,,,
As a farmer I can CRUD plots,,,,,,,,,,,,
As a farmer with a plot I can edit the parameters of a plot (current season),,,,,,,,,,,,
As a farmer with a plot I can sort my plots in the list,,,,,,,,,,,,
As a farmer with a plot I can filter my plots in the list,,,,,,,,,,,,
As a farmer with a plot I can export observation data for a plot,,,,,,,,,,,,
As a farmer with a plot I know when my plots will be in danger,,,,,,,,,,,,
As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,,,,,,,,,,,,
As a farmer irrigator I choose my preferred irrigation unit in my user-settings,,,,,,,,,,,,


### MR x Change : mrs2changes

In [43]:
mrs2changes = my_array = np.array(ctxRelPerCtxForms(df_changes, df_changesPerMR))

In [44]:
df_mrs2changes = pd.DataFrame(my_array, columns=df_changes["changes"], index=df_MRs["mrs"])
df_mrs2changes.to_csv("CSVs/ctxR_" + "mrs2changes" + "_Example.csv")

df_mrs2changes

changes,-7467257910948230577,-2369461973268875310,4830427670682130528,4532039579069182754,-3247514657019673527,-6143752595892114764,7361577847672698194,-606811720574785494,8505594899342993216,1522805805735262404,...,3661700103020512783,-5476404769532837106,-6935385260835073677,-4598298317307740577,-2317302283732295464,-509010681288107727,8500612336164843555,8638000273218908380,-9005986513360500478,5528272086623849427
mrs,,,,,,,,,,,,,,,,,,,,,
1,x,x,,,,,,,,,...,,,,,,,,,,
2,,,x,x,,,,,,,...,,,,,,,,,,
3,,,,,x,x,,,,,...,,,,,,,,,,
4,,,,,,,x,x,,,...,,,,,,,,,,
5,,,,,,,,,x,x,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,,,,,,,
7,,,,,,,,,,,...,,,,,,,,,,
8,,,,,,,,,,,...,x,x,,,,,,,,
9,,,,,,,,,,,...,,,x,x,,,,,,


### Change x File : changes2filenames

In [45]:
changes2filenames = my_array = np.array(ctxRelPerCtxForms(df_filenames, df_filenamesPerChanges))

In [46]:
df_changes2filenames = pd.DataFrame(my_array, columns=df_filenames["filenames"], index=df_changes["changes"])
df_changes2filenames.to_csv("CSVs/ctxR_" + "changes2filenames" + "_Example.csv")

df_changes2filenames.sort_values(by="changes")

filenames,src/user-settings,src/alert,src/plotList,src/danger,src/observation,src/export,src/admin,src/weather,src/recommendation,src/plot,src/farmer,src/irrigation
changes,,,,,,,,,,,,
-9005986513360500478,,,,,,,,x,,,,
-7467257910948230577,,,,,,,,,,,x,
-6935385260835073677,,,,,,,,,,,,x
-6143752595892114764,,,,,,,,,,x,,
-6072351681716626404,,,,,,x,,,,,,
-5476404769532837106,,,,,,,,,x,,,
-4598298317307740577,x,,,,,,,,,,,
-3247514657019673527,,,,,,,,,,,x,
-2369461973268875310,,,,,,,,x,,,,


### Diff x Change : changes2diffs

In [47]:
changes2diffs = my_array = np.array(ctxRelPerCtxForms(df_editTypes, df_editTypesPerChanges))

In [48]:
df_changes2diffs = pd.DataFrame(my_array, columns=df_editTypes["editTypes"], index=df_changes["changes"])
df_changes2diffs.to_csv("CSVs/ctxR_" + "changes2diffs" + "_Example.csv")

df_changes2diffs

editTypes,new_file,renamed_file,deleted_file,edited_file
changes,,,,
-7467257910948230577,,,,x
-2369461973268875310,x,,,
4830427670682130528,,,,x
4532039579069182754,,,,x
-3247514657019673527,,,,x
-6143752595892114764,,,,x
7361577847672698194,,,,x
-606811720574785494,,,,x
8505594899342993216,x,,,


## RCFT
RCFT file format is used to represent Relational Context Family: https://www.lirmm.fr/fca4j/ImportwithFamily.html

In [49]:
## A set of fonction used to format contexts to RCFT

def matrixFromFCtx(df_formalCtx, title):
    my_array = np.array(ctxRelPerCtxForms(df_formalCtx, df_formalCtx))
    df_formalCtxMatrixed = pd.DataFrame(my_array, columns=df_formalCtx[title], index=df_formalCtx[title])
    return df_formalCtxMatrixed


def printFormatdfCxtFToRCFT(df, title):
    print("FormalContext " + title)
    print("||")
    for index, row in df.iterrows():
        print("|" + str(row[0]) + "|")
    print()


def printFormatdfCxtRToRCFT(df, title, source, target):
    print("RelationalContext " + title)
    columns = list(df.columns.values)
    print("source " + source)
    print("target " + target)
    print("scaling exist")
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    print(strCol)
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        print(strtmp)
    print()


def retFormatdfCxtFToRCFT(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    strRet += ("||") + "\n"
    for index, row in df.iterrows():
        strRet += ("|" + str(row[0]) + "|") + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtRToRCFT(df, title, source, target):
    print(title, source, target)

    strRet = ""
    strRet += ("RelationalContext " + title) + "\n"
    columns = list(df.columns.values)
    strRet += ("source " + source) + "\n"
    strRet += ("target " + target) + "\n"
    strRet += ("scaling exist") + "\n"
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtMatrixed(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    columns = list(df.columns.values)
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet



In [50]:
## cleaning
df_MRs["mrs"] = df_MRs["mrs"].apply(lambda x : str(x))
df_changes["changes"] = df_changes["changes"].apply(lambda x : str(x))

### Contexts to RCFT
Building the RCFT file 

In [51]:
strRCTF = ""
#Formal Contexts
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_products,"products"),"products")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_US, "US"),"US")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_roles, "roles"),"roles")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_features, "features"),"features")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_MRs, "mrs"),"mrs")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_changes, "changes"),"changes")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_filenames, "filenames"),"filenames")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_editTypes, "editTypes"),"editTypes")
strRCTF += retFormatdfCxtMatrixed(matrixFromFCtx(df_clusters, "clusters"),"clusters")


In [52]:

#Relational Contexts
strRCTF += retFormatdfCxtRToRCFT(df_products2UserStories.transpose() , "products2UserStories", "products", "US")
strRCTF += retFormatdfCxtRToRCFT(df_userStories2Roles , "userStories2Roles", "US", "roles")                                 
strRCTF += retFormatdfCxtRToRCFT(df_userStories2features , "userStories2features", "US", "features")
strRCTF += retFormatdfCxtRToRCFT(df_features2AbstracFeatures , "features2AbstracFeatures", "features", "clusters")
strRCTF += retFormatdfCxtRToRCFT(df_userStories2mrs, "userStories2mrs", "US", "mrs")
strRCTF += retFormatdfCxtRToRCFT(df_mrs2changes , "mrs2changes", "mrs", "changes")
strRCTF += retFormatdfCxtRToRCFT(df_changes2filenames , "changes2filenames","changes", "filenames")
strRCTF += retFormatdfCxtRToRCFT(df_changes2diffs , "changes2diffs", "changes", "editTypes")

products2UserStories products US
userStories2Roles US roles
userStories2features US features
features2AbstracFeatures features clusters
userStories2mrs US mrs
mrs2changes mrs changes
changes2filenames changes filenames
changes2diffs changes editTypes


In [53]:
# Write rcft file
f = open("outputsRCFT/EXAMPLE_RCFT.rcft", "w")
f.write(strRCTF)
f.close()

### FCA4J: RCFT to JSON
The JSON is readable with RCAviz 

In [54]:
bashCommand = "java -jar fca4j-cli-0.4.4.jar RCA outputsRCFT/EXAMPLE_RCFT.rcft -a ARES outputsRCFT/outputFCA4J -fe -fi"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

## RCA for Feature Model

**Generation of the the extended family (last contexts including the relational attributes).
HERMES is an algorithm which computes the AOC-poset. We can alternatively use here another algorithm, e.g. to compute the whole lattice.**  
java -jar ./fca4j-cli-light-0.4.4.jar RCA -e -ra -f RCFT EXAMPLE_RCFT.rcft result/ -a HERMES  

**Format in CEX**  
java -jar ./fca4j-cli-light-0.4.4.jar FAMILY -a EXPORT result/EXAMPLE_RCFTextended.rcft -n products -x CEX result/products_extended.cex  

**The next step is to compute the rules between attributes for each extended formal context. This is done using the RULEBASIS command.**  
java -jar ./fca4j-cli-light-0.4.4.jar RULEBASIS result/products_extended.cex -folder result/products_rules


In [55]:

bashCommand = "java -jar fca4j-cli-0.4.4.jar RCA -e -ra -f RCFT result/EXAMPLE_RCFTextended.rcft result/ -a HERMES "
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

print("\nOutput: ", output.decode())


Output:  execute HERMES
computing ctx products 3x20 .
ctx products computed.
computing ctx US 12x55 .
ctx US computed.
computing ctx roles 4x4 .
ctx roles computed.
computing ctx features 12x17 .
ctx features computed.
computing ctx mrs 12x46 .
ctx mrs computed.
computing ctx changes 24x40 .
ctx changes computed.
computing ctx filenames 12x12 .
ctx filenames computed.
computing ctx editTypes 4x4 .
ctx editTypes computed.
computing ctx clusters 5x5 .
ctx clusters computed.
products: step=1 nb.concepts=7 nb.attrib rel.=0

US: step=1 nb.concepts=17 nb.attrib rel.=0

roles: step=1 nb.concepts=4 nb.attrib rel.=0

features: step=1 nb.concepts=15 nb.attrib rel.=0

mrs: step=1 nb.concepts=24 nb.attrib rel.=0

changes: step=1 nb.concepts=34 nb.attrib rel.=0

filenames: step=1 nb.concepts=12 nb.attrib rel.=0

editTypes: step=1 nb.concepts=4 nb.attrib rel.=0

clusters: step=1 nb.concepts=5 nb.attrib rel.=0

time:55 ms

ctx products : 3 entities, 37 attributes
ctx US : 12 entities, 98 attributes


In [56]:
bashCommand = "java -jar fca4j-cli-0.4.4.jar FAMILY -a EXPORT result/EXAMPLE_RCFTextended.rcft -n products -x CEX result/products_extended.cex"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#print("\nOutput: ", output.decode())

In [57]:
bashCommand = "java -jar fca4j-cli-0.4.4.jar RULEBASIS result/products_extended.cex -folder result/products_rules"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

#print("\nOutput: ", output.decode())

### Rules per support level

In [58]:
supportRulesProducts = output.decode().split("\n")[2:-1]
for rule in supportRulesProducts:
    print(rule,"\n")

<3>  => exist_products2UserStories(As a farmer I can CRUD plots),exist_products2UserStories(As a admin I can relaunch all failed simulation),exist_products2UserStories(exist_userStories2Roles(farmer)),exist_products2UserStories(exist_userStories2Roles(administrator)) 

<2> exist_products2UserStories(As a farmer I can CRUD plots),exist_products2UserStories(As a admin I can relaunch all failed simulation),exist_products2UserStories(exist_userStories2Roles(farmerirrigator)),exist_products2UserStories(exist_userStories2Roles(farmer)),exist_products2UserStories(exist_userStories2Roles(administrator)) => exist_products2UserStories(As a farmer irrigator I can view my irrigation recommendations in my favorite unit) 

<2> exist_products2UserStories(As a farmer I can CRUD plots),exist_products2UserStories(As a admin I can relaunch all failed simulation),exist_products2UserStories(exist_userStories2Roles(farmerwithplot)),exist_products2UserStories(exist_userStories2Roles(farmer)),exist_products2U

In [59]:
def get_deepest_content(s):
    pattern = r'\(([^()]*)\)'
    matches = re.findall(pattern, s)
    if matches:
        deepest_content = matches[-1]
    else:
        deepest_content = s
    return deepest_content


def cleanExist(implisList):
    cleanedImplis = []
    for imply in implisList:
        imply = imply.replace("\n","")
        cleanedImplis.append(get_deepest_content(imply))
    return cleanedImplis

def unique_strings(list_of_lists):
    return sorted(list(set([string for lst in list_of_lists for string in lst])))

def fillCtxRCA(df, row_list, col_list, value):
    for rowname in row_list:
        for colname in col_list:
            if df.loc[rowname][colname] != 2:
                df.loc[rowname][colname] = value
    return df

def getLogicperSupportLvl(df_emptyCol, ruleslist):
    premissesList = []
    implisList = []
    couplesListPremImp = []
    for row in ruleslist:
        implis = cleanExist(row.split("=>")[1].split(","))
        premisses = row.split("=>")[0]
        lvl = row.split("=>")[0][1:2]
        
        # if premisses are empty because they all are implies 
        if (lvl == str(len(df_emptyCol))):
            premisses = df_emptyCol.index.to_list()
            lvl = 2
        elif (lvl == str(0)):
            premisses = cleanExist(row.split("=>")[0][4:].split(","))
            implis = cleanExist(row.split("=>")[1].split(","))
            lvl = 0
        else: 
            premisses = (cleanExist(row.split("=>")[0][4:].split(",")))
            lvl = 1
            
            
        premissesList.append(premisses)
        implisList.append(implis)
        couplesListPremImp.append((lvl, premisses, implis)) 
    return couplesListPremImp

def relationnalCtxFromRCA(df_emptyCol, ruleslist):
    premissesList = []
    implisList = []
    couplesListPremImp = []
    for row in ruleslist:
        implis = cleanExist(row.split("=>")[1].split(","))
        premisses = row.split("=>")[0]
        lvl = row.split("=>")[0][1:2]
        
        # if premisses are empty because they all are implies 
        if (lvl == str(len(df_emptyCol))):
            premisses = df_emptyCol.index.to_list()
            lvl = 2
        elif (lvl == str(0)):
            premisses = []     #["toNegate"]+(cleanExist(row.split("=>")[0][4:].split(",")))
            implis = []        #cleanExist(row.split("=>")[1].split(","))
            lvl = -1
        else: 
            premisses = (cleanExist(row.split("=>")[0][4:].split(",")))
            lvl = 1
            
            
        premissesList.append(premisses)
        implisList.append(implis)
        couplesListPremImp.append((lvl, premisses, implis))    
            
    df_ctx = pd.DataFrame(index = unique_strings(premissesList), columns= unique_strings(implisList))#.fillna(0)
    #print("PREMISSES: ", df_ctx.head().index)
    #print("IMPLIES: ", df_ctx.data.columns)
    
    for lvl, premisses, implis in couplesListPremImp:
        df_ctx = fillCtxRCA(df_ctx, premisses, implis, lvl)
    return df_ctx.fillna(0)



In [60]:
df_RCA = df_products[["products"]].set_index("products")
df_logical = relationnalCtxFromRCA(df_RCA, supportRulesProducts)
df_logical


,Almond,Orchard,Vine,As a admin I can CRUD a farmer,As a admin I can relaunch all failed simulation,As a farmer I can refresh the predicted weather,As a farmer I can CRUD plots,As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,As a farmer irrigator I can view my irrigation recommendations in my favorite unit,As a farmer irrigator I choose my preferred irrigation unit in my user-settings,As a farmer with a plot I can export observation data for a plot,As a farmer with a plot I can filter my plots in the list,As a farmer with a plot I can sort my plots in the list,As a farmer with a plot I know when my plots will be in danger,administrator,current season,farmer,farmerirrigator,farmerwithplot
Almond,0,0,0,0,2,0,2,1,1,0,0,0,0,0,2,0,2,1,0
As a admin I can CRUD a farmer,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,1,0,0,1
As a admin I can relaunch all failed simulation,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1
As a farmer I can refresh the predicted weather,0,0,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1
As a farmer I can CRUD plots,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1
As a farmer irrigator I can manage my irrigations and recommendations in my favorite unit,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
As a farmer irrigator I can view my irrigation recommendations in my favorite unit,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
As a farmer irrigator I choose my preferred irrigation unit in my user-settings,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,1
As a farmer with a plot I can export observation data for a plot,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,1,0,1,1
As a farmer with a plot I can filter my plots in the list,0,1,0,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1


### Formated cross tree constraints

In [61]:
bashCommand = "java -jar  fca4j-cli-0.4.4.jar RULEBASIS result/products_extended.cex -folder result/features_rules"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()


supportRulesFeatures = output.decode().split("\n")[2:-1]
getLogicperSupportLvl(df_RCA, supportRulesFeatures)

[(2,
  ['Almond', 'Vine', 'Orchard'],
  ['As a farmer I can CRUD plots',
   'As a admin I can relaunch all failed simulation',
   'farmer',
   'administrator']),
 (1,
  ['As a farmer I can CRUD plots',
   'As a admin I can relaunch all failed simulation',
   'farmerirrigator',
   'farmer',
   'administrator'],
  ['As a farmer irrigator I can view my irrigation recommendations in my favorite unit']),
 (1,
  ['As a farmer I can CRUD plots',
   'As a admin I can relaunch all failed simulation',
   'farmerwithplot',
   'farmer',
   'administrator'],
  ['As a farmer  I can refresh the predicted weather',
   'current season',
   'As a farmer with a plot I know when my plots will be in danger',
   'As a admin I can CRUD a farmer']),
 (0,
  ['As a farmer I can CRUD plots',
   'As a admin I can relaunch all failed simulation',
   '9',
   'farmer',
   'administrator'],
  [' Orchard',
   'Almond',
   'Vine',
   'As a farmer  I can refresh the predicted weather',
   'current season',
   'As a farm

In [62]:
def apply_xor_ruleXML(strings):
    n = len(strings)
    if n == 1:
        return f'<var>{strings[0]}</var>'
    elif n == 2:
        return f'<not><eq><var>{strings[0]}</var><var>{strings[1]}</var></eq></not>'
    else:
        first = strings[0]
        second = strings[1]
        rest = strings[2:]
        sub_expr = apply_xor_ruleXML(rest)
        return f'<not><eq><var>{first}</var><not><eq><var>{second}</var>{sub_expr}</eq></not></eq></not>'


def apply_xor_rule(strings):
    n = len(strings)
    if n == 1:
        return strings[0]
    elif n == 2:
        return f'not ({strings[0]} iff {strings[1]})'
    else:
        first = strings[0]
        second = strings[1]
        rest = strings[2:]
        return apply_xor_rule([f'not ({first} iff not ({second} iff {apply_xor_rule(rest)}))'])

    
def apply_NotAandB_rule(rules):
    strAnd = "<not><conj>"
    for rule in rules:
        strAnd += "<var>"+ rule + "</var>"
    strAnd += "</conj></not>" 
    return strAnd
    
def cleanPremisslist(premiss, features, abstractF, df_USF):
    cleanedList = []
    for elem in premiss:
        

        #print(elem)
        if (elem in df_USF.index):
            df_f2usOneCol = df_USF.transpose()[[elem]]
            elem = df_f2usOneCol.index[df_f2usOneCol[elem] == "x"][0]
        if (elem in features or elem  in abstractF):
            cleanedList.append('"'+elem+'"')
    return cleanedList
    
featuresList = df_features["features"].tolist()
AbstractFList = df_clusters["clusters"].tolist()
crossTreeXor = []
for lvl, premisses, conclusion in getLogicperSupportLvl(df_RCA, supportRulesFeatures):
    if (lvl == 0):
        premisses  = cleanPremisslist(premisses, featuresList, AbstractFList,df_userStories2features)
        crossTreeXor.append(apply_NotAandB_rule(premisses))
        

for rule in crossTreeXor:
    print(rule.replace('"',""),"\n\n")

<not><conj><var>can CRUD plots</var><var>can relaunch all failed simulation</var></conj></not> 


<not><conj><var>can refresh the predicted weather</var><var>can CRUD plots</var><var>can export observation data for a plot</var><var>know when my plots will be in danger</var><var>can manage my irrigations and recommendations in my favorite unit</var><var>choose my preferred irrigation unit in my user-settings</var><var>can view my irrigation recommendations in my favorite unit</var><var>can CRUD a farmer</var><var>can relaunch all failed simulation</var></conj></not> 


<not><conj><var>can refresh the predicted weather</var><var>can CRUD plots</var><var>can sort my plots in the list</var><var>can filter my plots in the list</var><var>can export observation data for a plot</var><var>know when my plots will be in danger</var><var>choose my preferred irrigation unit in my user-settings</var><var>can view my irrigation recommendations in my favorite unit</var><var>can CRUD a farmer</var><var

In [63]:
def get_rows(df):
    min_zeros = df.apply(lambda x: sum(x == 0), axis=1).min()
    return df[df.apply(lambda x: sum(x == 0), axis=1) == min_zeros]


## FM synthesis
The XML generated could be used with featureIDE. It represents a feature model and a role feature model

### Feature Model

In [64]:
def FMformatXML(df_premissesconclusion, df_f2fa, crossTreeXor, title):
    properties = '\t<properties>\n \t\t<graphics key="legendautolayout" value="true"/>\n \t\t<graphics key="showshortnames" value="false"/>\n\t\t<graphics key="layout" value="vertical"/>\n\t\t<graphics key="showcollapsedconstraints" value="true"/>\n\t\t<graphics key="legendhidden" value="false"/>\n\t\t<graphics key="layoutalgorithm" value="4"/>\n\t</properties>\n'

    strfm=('<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<featureModel>\n')
    strfm+=(properties)
    strfm+=('<struct>\n')
    strfm+=('\t<and abstract="true" mandatory="true" name="'+title+'">\n')
    for column_name in df_f2fa.columns:
        if column_name != "":
            strfm+=('\t\t<and abstract="true" name="'+column_name+'">\n')
            for row_name in list(df_f2fa.loc[df_f2fa[column_name] == 'x'].index):
                try:
                    isMandatory = df_premissesconclusion.loc[row_name,column_name] == 2
                except KeyError:
                    #print(column_name, row_name)
                    isMandatory = False
                except AttributeError:
                    isMandatory = False

                strfm+=('\t\t\t<feature mandatory="' + str(isMandatory) +  '" name="'+row_name+'"/>\n')
        strfm+=('\t\t</and>\n')
    strfm+=('\t</and>\n</struct>\n')
    
    strfm+=('\t<constraints>\n')
    for rule in crossTreeXor:
        strfm+= "\t\t<rule>\n"
        strfm+= rule.replace('"',"") + "\n"
        strfm+= "\t\t</rule>\n"    
    strfm+= "\t</constraints>"
        
    strfm+=("</featureModel>\n")
    f = open("FMExample.txt", "w")
    f.write(strfm)
    f.close()
    return strfm

print(FMformatXML(df_logical, df_features2AbstracFeatures, crossTreeXor, "Example"))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<featureModel>
	<properties>
 		<graphics key="legendautolayout" value="true"/>
 		<graphics key="showshortnames" value="false"/>
		<graphics key="layout" value="vertical"/>
		<graphics key="showcollapsedconstraints" value="true"/>
		<graphics key="legendhidden" value="false"/>
		<graphics key="layoutalgorithm" value="4"/>
	</properties>
<struct>
	<and abstract="true" mandatory="true" name="Example">
		<and abstract="true" name="refresh predicted weather">
			<feature mandatory="False" name="can refresh the predicted weather"/>
		</and>
		<and abstract="true" name="crud plots">
			<feature mandatory="False" name="can CRUD plots"/>
		</and>
		<and abstract="true" name="plots plot list">
			<feature mandatory="False" name="can edit the parameters of a plot current season "/>
			<feature mandatory="False" name="can sort my plots in the list"/>
			<feature mandatory="False" name="can filter my plots in the list"/>
			<feature mandatory

#### Feature model visualized with FeatureIDE
![exampleFMCONO.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/exampleFMCONO.png)


### Feature role model

In [65]:
def metaFMformatXML(df_premissesconclusion, df_f2fa, df_rolesFeaturesCluster, crossTreeXor, title):
    properties = '\t<properties>\n \t\t<graphics key="legendautolayout" value="true"/>\n \t\t<graphics key="showshortnames" value="false"/>\n\t\t<graphics key="layout" value="vertical"/>\n\t\t<graphics key="showcollapsedconstraints" value="true"/>\n\t\t<graphics key="legendhidden" value="false"/>\n\t\t<graphics key="layoutalgorithm" value="4"/>\n\t</properties>\n'

    strfm=('<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<featureModel>\n')
    strfm+=(properties)
    strfm+=('<struct>\n')
    strfm+=('\t<and abstract="true" mandatory="true" name="'+title+'">\n')
    
    roles = list(dict.fromkeys(df_features2Roles["roleCluster"].tolist()))

    for role in roles:
        featsRoles = df_features2Roles[df_features2Roles['roleCluster'] == role].index.tolist()
        strfm+=('\t\t<and abstract="true" name="'+role+'">\n')
        for column_name in df_f2fa.loc[featsRoles].replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all').columns:
            if column_name != "":
                strfm+=('\t\t\t<and abstract="true" name="'+column_name+'">\n')
                for row_name in list(df_f2fa.loc[df_f2fa[column_name] == 'x'].index):
                    try:
                        isMandatory = df_premissesconclusion.loc[row_name,column_name] == 2
                    except KeyError:
                        isMandatory = False
                    except AttributeError:
                        isMandatory = False

                    strfm+=('\t\t\t\t<feature mandatory="' + str(isMandatory) +  '" name="'+row_name+'"/>\n')
            strfm+=('\t\t\t</and>\n')
        strfm+=('\t\t</and>\n')
    
    strfm+=('\t</and>\n</struct>\n')
    
    strfm+=('\t<constraints>\n')
    for rule in crossTreeXor:
        strfm+= "\t\t<rule>\n"
        strfm+= rule.replace('"',"") + "\n"
        strfm+= "\t\t</rule>\n"    
    strfm+= "\t</constraints>\n"
        
    strfm+=("</featureModel>\n")
    f = open("RoleFeatureModelExample.txt", "w")
    f.write(strfm)
    f.close()
    return strfm
print(metaFMformatXML(df_logical, df_features2AbstracFeatures, df_features2Roles, crossTreeXor, "RoleFeatureModelExample"))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<featureModel>
	<properties>
 		<graphics key="legendautolayout" value="true"/>
 		<graphics key="showshortnames" value="false"/>
		<graphics key="layout" value="vertical"/>
		<graphics key="showcollapsedconstraints" value="true"/>
		<graphics key="legendhidden" value="false"/>
		<graphics key="layoutalgorithm" value="4"/>
	</properties>
<struct>
	<and abstract="true" mandatory="true" name="RoleFeatureModelExample">
		<and abstract="true" name="farmer">
			<and abstract="true" name="refresh predicted weather">
				<feature mandatory="False" name="can refresh the predicted weather"/>
			</and>
			<and abstract="true" name="crud plots">
				<feature mandatory="False" name="can CRUD plots"/>
			</and>
			<and abstract="true" name="plots plot list">
				<feature mandatory="False" name="can edit the parameters of a plot current season "/>
				<feature mandatory="False" name="can sort my plots in the list"/>
				<feature mandatory="False

#### Feature role model visualized with FeatureIDE
![exampleFRMCONO.png](https://raw.githubusercontent.com/Hyrlos/CoNoConcepts_ThomasGEORGES_LIRMM/master/exampleFRMCONO.png)